# GT95 revisited


In this note, I would like to revisit GT95 and to show how to "automatically" derive the optimally accurate operators


In [ ]:

using Symbolics
using SymbolicUtils

## Homogeneous SH


In [ ]:
#function linearSpline(x::Float64,j::Int,N::Int,Δz::Float64)
function linearSpline(x,j,N,Δz)
#function linearSpline(x::SymbolicUtils.Sym{Real, Nothing}, j::SymbolicUtils.Sym{Int64, Nothing}, N::SymbolicUtils.Sym{Int64, Nothing}, xpoint::SymbolicUtils.Sym{SymbolicUtils.FnType{Tuple{Number}, Number}, Nothing})
    @syms xpoint(j)
    X_local=0.e0
    xpoint = j -> Real(j-1)*Δz
    if j-1>=0 && xpoint(j-1) <= x < xpoint(j)
        X_local = (x-xpoint(j-1))/(xpoint(j)-xpoint(j-1))
    elseif j+1<=N && xpoint(j) <= x <= xpoint(j+1)
        X_local = (xpoint(j+1)-x)/(xpoint(j+1)-xpoint(j))
    end

    return X_local
end


In [ ]:

@syms x::Real Δz::Real j::Int N::Int
#@syms xpoint[1:N+1]::Array{Real,1}
@syms X(j,x)::Real # spline functions 
N=10    
j=1:N
#xpoint = j -> Real(j-1)*Δz
X= (j,x) -> linearSpline(x,j,N,Δz)
#X=(j,x) ->sin(x)^j


In [ ]:
A[I_1, I_2, ..., I_n]=X

In [ ]:
X=linearSpline(1.0,1,10,1.0)


In [ ]:
typeof(X(j,x))

# OK do this differently

OK, let's forget about making an array of functions with Symbolics, it's been already more than one week!

In [ ]:
function zerothSpline(x,j,nodes)
    X_local=0.e0
    if j>=1 && j+1 <=length(nodes) && nodes[j] <= x < nodes[j+1]
        X_local = 1.e0
    end
end

function Spline(x,j,k,nodes)
    X_local=0.e0
    if k>0 
        if j+1 <= length(nodes)
            X_local += (x-nodes[j])/(nodes[j+k]-nodes[j])*Spline(x,j,k-1,nodes) 
        end
        if j+k+1 <= length(nodes)
            X_local += X_local + (nodes[j+k+1]-x)/(nodes[j+k+1]-nodes[j+1])*Spline(x,j+1,k-1,nodes)
        end      
    elseif k==0
        if j>=1 && j+1 <=length(nodes) && nodes[j] <= x < nodes[j+1]
            X_local = 1.e0
        end
    end
end

In [ ]:
@variables  Δz::Real 
N=50 # number of nodes
K=3 # maximum order of spline
Nresolution =10 # number of points between nodes
Xspline=Array{Symbolics.Real,3}(undef,K+1+1,N+1,N*Nresolution+1);
XsplinePrime=Array{Symbolics.Real,3}(undef,K+1+1,N+1,N*Nresolution+1);


In [ ]:


#@variables Xspline[1:K+1,1:N+1,1:N*Nresolution+1] XsplinePrime[1:K+1,1:N+1,1:N*Nresolution+1];

nodes=[Δz*(i-1) for i in 1:N+1]
xarray=[Δz*(i-1)//Nresolution for i in 1:N*Nresolution+1]
#Xspline=zeros(N+1,K+1,1:N*Nresolution+1)



In [ ]:

for k = 0:K # k is shifted after when stored (but not now in this routine) with respect to the ordering of spline functions
    for j = 1:N+1 # j is shifted by 1 compared to the paper
      for xi= 1:N*Nresolution+1 # xi is shifted by 1 
        #X_local = Spline(x[xi],j,k,nodes)
        x=xarray[xi]
        X_local=0
        X_local_prime=0
        if k>0 
            if j+k <= N+1 
                
                X_local += (x-nodes[j])/(nodes[j+k]-nodes[j])*Xspline[k-1+1,j,xi]
                X_local_prime +=  (x-nodes[j])/(nodes[j+k]-nodes[j])*XsplinePrime[k-1+1,j,xi] + Xspline[k-1+1,j,xi]/(nodes[j+k]-nodes[j])
            end
            if j+k+1 <= N+1
                X_local +=  (nodes[j+k+1]-x)/(nodes[j+k+1]-nodes[j+1])*Xspline[k-1+1,j+1,xi]
                X_local_prime += -1/(nodes[j+k+1]-nodes[j+1])*Xspline[k-1+1,j+1,xi] + (nodes[j+k+1]-x)/(nodes[j+k+1]-nodes[j+1])*XsplinePrime[k-1+1,j+1,xi]
            end      
        elseif k==0
            if j>=1 && j+1 <= N+1 && nodes[j]/Δz <= x/Δz < nodes[j+1]/Δz
                X_local = 1
            end
        end
        Xspline[k+1,j,xi]=X_local
        XsplinePrime[k+1,j,xi]=X_local_prime
      end
    end
end



In [ ]:
XsplinePrime[2,2,2]



In [ ]:
Xspline[1,3,24]

In [ ]:
@variables A[1:5, 1:3] b[1:3]

In [ ]:
Symbolics.scalarize(A[2, 1]) = 1